In [ ]:
## 基础特征计算
import pandas as pd
import glob
from utils.basicFeature import basicFeature

csv_files = glob.glob("./time-series-data/*.csv")
features_list = []
for file in csv_files:
    # print(file)
    features_list.append(basicFeature.integrate_all_features(file))

features_df = pd.concat(features_list)
print(features_df)
    

In [ ]:
## 基础特征可视化与个股分析
import matplotlib.pyplot as plt
selected = ['AAPL', 'GE', 'MCD', 'AABA', 'NKE', 'WMT']
df_sel = features_df.loc[selected]

plt.figure(figsize=(8,6))
plt.scatter(df_sel['Volatility'], df_sel['AvgReturn'], c='blue', s=100)

for i in df_sel.index:
    plt.text(df_sel.loc[i, 'Volatility'] + 0.0005, df_sel.loc[i, 'AvgReturn'], i)

plt.xlabel("Volatility (Risk)")
plt.ylabel("Average Daily Return")
plt.title("Return vs. Volatility for Selected Stocks")
plt.grid(True)
plt.show()

In [ ]:
df_sel['SharpeRatio'].sort_values().plot(kind='barh', figsize=(7,5), color='green')
plt.xlabel("Sharpe Ratio")
plt.title("Sharpe Ratio of Selected Stocks")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
df_sel['MDD'].sort_values().plot(kind='barh', color='red')
plt.title("Max Drawdown of Selected Stocks")
plt.xlabel("Max Drawdown")
plt.grid(True)
plt.show()

In [ ]:
## 高级指标计算
import pandas as pd
import glob
from utils.advancedFeature import advancedFeature

csv_files = glob.glob("./time-series-data/*.csv")
advanced_features_list = []
for file in csv_files:
    # print(file)
    advanced_features_list.append(advancedFeature.integrate_advanced_features(file))

advanced_features_df = pd.concat(advanced_features_list)
print(advanced_features_df)

In [ ]:
## 日利率可视化：以AABA为例
df_AABA = advancedFeature.load_data("./time-series-data/AABA_2006-01-01_to_2018-01-01.csv")
df_AABA['Return'] = df_AABA['Close'].pct_change()
plt.figure(figsize=(6,4))
df_AABA['Return'].hist(bins=50, alpha=0.7, color='skyblue')
plt.title("Daily Return Distribution")
plt.xlabel("Daily Return")
plt.ylabel("Frequency")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
## 布林带可视化：以AABA为例
import matplotlib.dates as mdates

df_AABA['MA20'] = df_AABA['Close'].rolling(window=20).mean()
df_AABA['STD20'] = df_AABA['Close'].rolling(window=20).std()
df_AABA['Upper'] = df_AABA['MA20'] + 2 * df_AABA['STD20']
df_AABA['Lower'] = df_AABA['MA20'] - 2 * df_AABA['STD20']
df_AABA['Date'] = pd.to_datetime(df_AABA['Date'])

plt.figure(figsize=(10,5))
plt.plot(df_AABA['Date'], df_AABA['Close'], label='Close', color='black', linewidth=1)
plt.plot(df_AABA['Date'], df_AABA['MA20'], label='MA20', color='blue')
plt.plot(df_AABA['Date'], df_AABA['Upper'], label='Upper Band', linestyle='--', color='red')
plt.plot(df_AABA['Date'], df_AABA['Lower'], label='Lower Band', linestyle='--', color='green')
plt.fill_between(df_AABA['Date'], df_AABA['Upper'], df_AABA['Lower'], color='orange', alpha=0.3)

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

plt.title("Bollinger Bands")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
## RSI可视化：以AABA为例
df = advancedFeature.load_data("./time-series-data/AABA_2006-01-01_to_2018-01-01.csv")
delta = df['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df['RSI14'] = 100 - (100 / (1 + rs))

df['Date'] = pd.to_datetime(df['Date'])

plt.figure(figsize=(10,4))
plt.plot(df['Date'], df['RSI14'], label='RSI (14)', color='purple')

ax = plt.gca()
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

plt.axhline(70, linestyle='--', color='red', label='Overbought')
plt.axhline(30, linestyle='--', color='green', label='Oversold')
plt.title("RSI 14 Indicator")
plt.xlabel("Date")
plt.ylabel("RSI")
plt.legend()
plt.tight_layout()
plt.show()